### Import Statements

In [3]:
import numpy as np
import scipy as sp
import json
import csv
from sklearn.feature_extraction import text
import matplotlib.pyplot as plt

## Classes

In [13]:
# Dataset constants ==============================
N = 331675   # Number of points
D = 7   # Number of (useful) features
C = 172    # Number of unique categories
mC = 1508    # Largest label coding (plus one for array creation purposes)
M = 12    # Number of unique months

class NaiveBayes(object):
    
    def __init__(self):
        # Load test file
        csv_file = open('KickStarterData_nb.csv', encoding="utf-8")
        csv_reader = csv.reader(csv_file, delimiter=',')
        
        data_arr = np.empty((N, D + 1))    # D + 1 since need to account for the labels
        counter = -2    # For skipping the row of feature names 
        for row in csv_reader:
            counter += 1
            if (counter == -1):
                continue
                            
            temp_arr = np.array(row)
            data_arr[counter] = temp_arr[1:]    # Ignores the ID vaules
        
        # Splits the data between data points and labels
        self.x = data_arr[:, 0:D]
        self.y = data_arr[:, D]
        
        # Set all other instance variables to None
        self.priors = None
        self.likelihoods_cont_0 = None
        self.likelihoods_cont_1 = None
        self.likelihood_cat_0 = None
        self.likelihood_cat_1 = None
        self.likelihood_mon_0 = None
        self.likelihood_mon_1 = None
        
    def find_priors(self, y_train):
        """
        ARGS:
            y_train: (N,) numpy array, labels for the points in the training set
        SETS:
            self.priors: (2,) numpy array, the probabilities of having a label of 0 or 1, P(y)
        """
        # Get total number of labels
        N_train = np.shape(y_train)[0]
        
        # Get the number of each label
        num_1 = y_train.sum()
        num_0 = N_train - num_1
        
        # Calculate each prior
        priors = np.array([num_0, num_1])
        priors = priors / N_train
        
        self.priors = priors

    def find_likelihoods_cont(self, x_train, y_train):
        """
        Finds the mean and variance of each continuous feature (so all features minus category) to be
        able to use a Guassian estimation.
        
        ARGS:
            x_train: (N, D) numpy array, data set for the model to be trained on
            y_train: (N,) numpy array, labels for x_train
        SETS:
            self.likelihoods_cont_0: (D - 2, 2) numpy array, the means ([:, 0]) and the variance
                                     ([:, 1]) of each of the D continuous features for y = 0, P(x|y = 0)
            self.likelihoods_cont_1: (D - 2, 2) numpy array, the means ([:, 0]) and the variance
                                     ([:, 1]) of each of the D continuous features for y = 1, P(x|y = 1)
        """
        # Transpose the labels to be vertical
        y_train_vert = np.array([y_train]).T
        
        # Get the continuous data points for each label
        x_train_1 = x_train * y_train_vert
        x_train_1 = x_train_1[~np.all(x_train_1 == 0, axis=1)]
        x_train_0 = x_train * (1 - y_train_vert)
        x_train_0 = x_train_0[~np.all(x_train_0 == 0, axis=1)]
        
        # Take out the ordinal data
        x_train_1 = x_train_1[:, 2:]
        x_train_0 = x_train_0[:, 2:]
        
        # Get the means and variances for each subset of the training set
        avg_0 = np.average(x_train_0, axis=0)
        var_0 = np.var(x_train_0, axis=0)
        avg_1 = np.average(x_train_1, axis=0)
        var_1 = np.var(x_train_1, axis=0)
        
        # Stack the means and variances to fit the (D, 2) shape
        hyper_params_0 = np.column_stack((avg_0, var_0))
        hyper_params_1 = np.column_stack((avg_1, var_1))
        
        self.likelihoods_cont_0 = hyper_params_0
        self.likelihoods_cont_1 = hyper_params_1
        
    def find_likelihoods_cat(self, x_train, y_train):
        """
        Finds the likelihood of each category. Uses Laplace Smoothing to avoid zeros, where the added value
        is 1/N, where N is the number of unique values. (Add-1 Smoothing is Laplace Smoothing when the added
        value is 1.)
        Wiki Article on Laplace Smooting: https://en.wikipedia.org/wiki/Additive_smoothing
        
        ARGS:
            x_train: (N, D) numpy array, data set for the model to be trained on
            y_train: (N,) numpy array, labels for x_train
        SETS:
            self.likelihoods_cat_0: (mC,) numpy array, the likelihood for each category for y = 0, P(x|y = 0)
            self.likelihoods_cat_1: (mC,) numpy array, the likelihood for each category for y = 1, P(x|y = 1)
        """
        # Get the number of each label
        num_1 = y_train.sum()
        num_0 = np.shape(y_train)[0] - num_1
        
        # Transpose the labels to be vertical
        y_train_vert = np.array([y_train]).T
        
        # Get the data points for each label
        x_train_1 = x_train * y_train_vert
        x_train_1 = x_train_1[~np.all(x_train_1 == 0, axis=1)]
        x_train_0 = x_train * (1 - y_train_vert)
        x_train_0 = x_train_0[~np.all(x_train_0 == 0, axis=1)]
        
        # Isolate the category IDs
        x_train_0 = x_train_0[:, 0]
        x_train_1 = x_train_1[:, 0]
        
        # Instantiate the likelihood arrays
        likelihood_cat_0 = np.zeros((mC,))
        likelihood_cat_1 = np.zeros((mC,))
        
        # Count the number of each category
        for val in x_train_0.astype(int):
            likelihood_cat_0[val] += 1
        for val in x_train_1.astype(int):
            likelihood_cat_1[val] += 1
            
        # Calculate smoothing value
        smoothing_alpha = 1 / C
        
        # Calculate likelihoods
        # Note that the sum will yield >1, but only because there are many unused values
        # If summing only the used values, will yield 1
        likelihood_cat_0 = (likelihood_cat_0 + smoothing_alpha) / (num_0 + 1)
        likelihood_cat_1 = (likelihood_cat_1 + smoothing_alpha) / (num_1 + 1)
        
        self.likelihood_cat_0 = likelihood_cat_0
        self.likelihood_cat_1 = likelihood_cat_1
    
    def find_likelihoods_mon(self, x_train, y_train):
        """
        Finds the likelihood of each month. Uses Laplace Smoothing to avoid zeros, where the added value
        is 1/N, where N is the number of unique values. (Add-1 Smoothing is Laplace Smoothing when the added
        value is 1.)
        Wiki Article on Laplace Smooting: https://en.wikipedia.org/wiki/Additive_smoothing
        
        ARGS:
            x_train: (N, D) numpy array, data set for the model to be trained on
            y_train: (N,) numpy array, labels for x_train
        SETS:
            self.likelihoods_mon_0: (M+1,) numpy array, the likelihood for each mon for y = 0, P(x|y = 0)
            self.likelihoods_mon_1: (M+1,) numpy array, the likelihood for each mon for y = 1, P(x|y = 1)
        """
        # Get the number of each label
        num_1 = y_train.sum()
        num_0 = np.shape(y_train)[0] - num_1
        
        # Transpose the labels to be vertical
        y_train_vert = np.array([y_train]).T
        
        # Get the data points for each label
        x_train_1 = x_train * y_train_vert
        x_train_1 = x_train_1[~np.all(x_train_1 == 0, axis=1)]
        x_train_0 = x_train * (1 - y_train_vert)
        x_train_0 = x_train_0[~np.all(x_train_0 == 0, axis=1)]
        
        # Isolate the months
        x_train_0 = x_train_0[:, 1]
        x_train_1 = x_train_1[:, 1]
        
        # Instantiate the likelihood arrays
        likelihood_mon_0 = np.zeros((M + 1,))
        likelihood_mon_1 = np.zeros((M + 1,))
        
        # Count the number of each category
        for val in x_train_0.astype(int):
            likelihood_mon_0[val] += 1
        for val in x_train_1.astype(int):
            likelihood_mon_1[val] += 1
            
        # Calculate smoothing value
        smoothing_alpha = 1 / M
        
        # Calculate likelihoods
        # Note that the sum will yield >1, but only because there are many unused values
        # If summing only the used values, will yield 1
        likelihood_cat_0 = (likelihood_mon_0 + smoothing_alpha) / (num_0 + 1)
        likelihood_cat_1 = (likelihood_mon_1 + smoothing_alpha) / (num_1 + 1)
        
        self.likelihood_mon_0 = likelihood_mon_0
        self.likelihood_mon_1 = likelihood_mon_1
        
    def train(self, x_train, y_train):
        """
        ARGS:
            x_train: (N, D) numpy array, data set for the model to be trained on
                Note: Feature 1 (Category) is ordinal and should be treated as such
                      All other features are continuous and should be treated as such (Guassian Distribution)
            y_train: (N,) numpy array, labels for x_train
        SET:
            All the instance variables (implicitly)
        NOTE:
            Since classification is based on relative values, the normalization constant
            (the denominator, P(X)) is not needed and does not need to be calculated
        """
        print("    Starting to find the priors...")
        self.find_priors(y_train)
        print("    Completed finding the priors...\n")
        
        print("    Starting to find the likelihood of the categories...")
        self.find_likelihoods_cat(x_train, y_train)
        print("    Completed finding the likelihood of the categories...\n")
        
        print("    Starting to find the likelihood of the months...")
        self.find_likelihoods_mon(x_train, y_train)
        print("    Completed finding the likelihood of the months...\n")
        
        print("    Starting to find the likelihoods of the rest of the features...")
        self.find_likelihoods_cont(x_train, y_train)
        print("    Completed finding the likelihoods of the rest of the features...")
        
    def test(self, x_test):
        """
        ARGS:
            x_test: (N, D) numpy array, data set for the model to be tested on
                Note: Feature 1 (Category) is ordinal and should be treated as such
                      All other features are continuous and should be treated as such (Guassian Distribution)
        RETURNS:
            y_pred: (N,) numpy array, predicted labels
        NOTE:
            Since classification is based on relative values, the normalization constant
            (the denominator, P(X)) is not needed and does not need to be calculated
        """
        # Get number of data points in the test set
        N_test, D_test = np.shape(x_test)
        
        # Probabilities of each label for each data point
        prob = np.zeros((N_test, 2))
        
        # Start with the priors, P(y)
        prob = prob + self.priors
        
        # Multiply the likelihood of the categories, P(x = category|y)
        likelihood_cat_0 = self.likelihood_cat_0[x_test[:, 0].astype(int)]
        likelihood_cat_1 = self.likelihood_cat_1[x_test[:, 0].astype(int)]
        likelihood_cat = np.column_stack((likelihood_cat_0, likelihood_cat_1))
        prob = prob * likelihood_cat
        
        # Multiply the likelihood of the months, P(x = month|y)
        likelihood_mon_0 = self.likelihood_mon_0[x_test[:, 1].astype(int)]
        likelihood_mon_1 = self.likelihood_mon_1[x_test[:, 1].astype(int)]
        likelihood_mon = np.column_stack((likelihood_mon_0, likelihood_mon_1))
        prob = prob * likelihood_mon
        
        # Multiply the probabilities from the continuous features, P(x = everything else|y)
        for i in range(2, D_test):
            curr_features = x_test[:, i]
            
            mean_0 = self.likelihoods_cont_0[i - 2, 0]
            std_dev_0 = np.sqrt(self.likelihoods_cont_0[i - 2, 1])
            mean_1 = self.likelihoods_cont_1[i - 2, 0]
            std_dev_1 = np.sqrt(self.likelihoods_cont_1[i - 2, 1])
            
            curr_features_prob_0 = sp.stats.norm.pdf(curr_features, mean_0, std_dev_0)
            curr_features_prob_1 = sp.stats.norm.pdf(curr_features, mean_1, std_dev_1)
            
            curr_features_prob = np.column_stack((curr_features_prob_0, curr_features_prob_1))
            
            prob = prob * curr_features_prob
        
        # Find predicted labels
        y_pred = np.argmax(prob, axis=1)
        return y_pred
    
    def calc_metrics(self, y_test, y_pred):
        """
        ARGS:
            y_test: (N,) numpy array, true labels of the test set
            y_pred: (N,) numpy array, predicted labels of the test set
        RETURNS:
            accuracy: float, accuracy of this Naive Bayes model
            precision: float, precision of this Naive Bayes model
            recall: float, recall of this Naive Bayes model
            f1: float, f1 score of this Naive Bayes model
        """
        # Get number of data points that were tested
        N_tested = np.shape(y_test)[0]
        
        # Get the results as ints
        y_test_int = y_test.astype(int)
        y_pred_int = y_pred.astype(int)
        
        # Get the wrong predictions
        wrong_preds = np.bitwise_xor(y_test_int, y_pred_int)
        
        # Find the true/false postives/negatives
        t_pos = np.bitwise_and(y_test_int, y_pred_int)
        f_pos = np.bitwise_and(y_pred_int, wrong_preds)
        t_neg = np.bitwise_and(1 - y_test_int, 1 - y_pred_int)
        f_neg = np.bitwise_and(y_test_int, wrong_preds)
        
        num_tp = t_pos.sum()
        num_fp = f_pos.sum()
        num_tn = t_neg.sum()
        num_fn = f_neg.sum()
        
        # Calculate metrics
        accuracy = (num_tp + num_tn) / (num_tp + num_fp + num_tn + num_fn)
        precision = num_tp / (num_tp + num_fp)
        recall = num_tp / (num_tp + num_fn)
        f1 = (precision * recall) / (precision + recall)
        
        return accuracy, precision, recall, f1

In [14]:
class Cleaning(object):
    #class to handle cleaning data (standardizing features, dimensionality reduction, etc.)
    
    #PCA for dimensionality reduction of the dataset
    def pca(self, X, var = False, k=2):
        """
        ARGS:
            X: (N X D) data set as a numpy array, uncentered
            var: whether to use retained variance or number of features as the basis of reduction
            k: if var=FALSE, k is the number of features to be kept. if var=TRUE, k is the retained variance as a decimal

        RETURNS:
            new_data: dataset (as a numpy array) obtained by applying PCA on the original dataset
        """

        #center the data set
        centeredData = X - np.mean(X, axis=0)
        __, S, V = np.linalg.svd(centeredData, full_matrices = False)

        #if var== False, do PCA based on the specified number of features
        if (var == False):
            #if k not entered correctly (less than one feature entered), assume default number of features
            if (k < 1):
                k = 2
            V = V[0:k,:]
            new_data = np.matmul(centeredData, V.T)
        
        #if var==True, do PCA based on the specified number of features
        else:
            #if k not entered correctly (greater than 100%), assume default variance
            if (k >= 1):
                k = .99
            new_data = np.matmul(centeredData, V.T)
            ssquared = np.square(S)
            origVar = np.sum(ssquared)
            for i in range(np.shape(centeredData)[1]):
                var = np.sum(ssquared[0:i])/origVar
                if (var >= k):
                    V_temp = V[0:i,:]
                    new_data= np.matmul(centeredData, V_temp.T)
                    break
        return new_data





## Training and Testing the Model

In [15]:
print("Starting to create the model...")
nb = NaiveBayes()
print("Completed creating the model...\n")

print("Starting to run PCA on the data...")
ord_columns = nb.x[:, 0:2]    # Isolate the odrinal data
clean = Cleaning()
cleaned_x = clean.pca(nb.x[:, 2:], k=4)    # Don't run ordinal data through PCA
cleaned_x = np.hstack((ord_columns, cleaned_x))
print("Completed running PCA on the data...")

# Test on 20% of the dataset
test_amount = N // 5
x_test = cleaned_x[0:test_amount, :]
y_test = nb.y[0:test_amount]
x_train = cleaned_x[test_amount:, :]
y_train = nb.y[test_amount:]

# Train the model
print("Starting to train the model...")
nb.train(x_train, y_train)
print("Completed training the model...\n")

# Test the model
print("Starting to test the model...")
y_pred = nb.test(x_test)
print("Completed testing the model...\n")

# Find metrics
accuracy, precision, recall, f1 = nb.calc_metrics(y_test, y_pred)
print("The model had an accuracy of %.4f." % accuracy)
print("The model had a precision of %.4f." % precision)
print("The model had a recall of %.4f." % recall)
print("The model had an f1 score of %.4f." % f1)

Starting to create the model...
Completed creating the model...

Starting to run PCA on the data...
Completed running PCA on the data...
Starting to train the model...
    Starting to find the priors...
    Completed finding the priors...

    Starting to find the likelihood of the categories...
    Completed finding the likelihood of the categories...

    Starting to find the likelihood of the months...
    Completed finding the likelihood of the months...

    Starting to find the likelihoods of the rest of the features...
    Completed finding the likelihoods of the rest of the features...
Completed training the model...

Starting to test the model...
Completed testing the model...

The model had an accuracy of 0.6892.
The model had a precision of 0.5739.
The model had a recall of 0.8804.
The model had an f1 score of 0.3474.
